<a href="https://colab.research.google.com/github/AmeyaKI/multimodal-asr/blob/main/asr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive, files
drive.mount('/content/drive')
import os

Mounted at /content/drive


In [8]:
%pip install jiwer
from jiwer import wer, cer
import torch
import numpy as np
from transformers import AutoProcessor, AutoModelForCTC
import librosa
import os
from pathlib import Path

In [30]:
desired_file = 'output_audio' # testing audio from peoples speech dataset

file_path = f"/content/drive/MyDrive/Colab Notebooks/data/asr/{desired_file}.wav"

if Path(file_path).is_file():
    print(f"{desired_file}.wav found")
else:
    print(f"{desired_file}.wav not found")

output_audio.wav found


In [19]:
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))

CUDA available: True
GPU name: Tesla T4


In [20]:
MODEL = 'nvidia/parakeet-ctc-1.1b' # train custom model ?
device = 'cuda'

processor = AutoProcessor.from_pretrained(MODEL)
model = AutoModelForCTC.from_pretrained(MODEL, dtype='auto')

model.eval()

preprocessor_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/634 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/962 [00:00<?, ?B/s]

🚨 Config not found for parakeet. You can manually add it to HARDCODED_CONFIG_FOR_MODELS in utils/auto_docstring.py
🚨 Config not found for parakeet. You can manually add it to HARDCODED_CONFIG_FOR_MODELS in utils/auto_docstring.py
🚨 Config not found for parakeet. You can manually add it to HARDCODED_CONFIG_FOR_MODELS in utils/auto_docstring.py


model.safetensors:   0%|          | 0.00/4.25G [00:00<?, ?B/s]

ParakeetForCTC(
  (encoder): ParakeetEncoder(
    (subsampling): ParakeetEncoderSubsamplingConv2D(
      (layers): ModuleList(
        (0): Conv2d(1, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (1): ReLU()
        (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=256)
        (3): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
        (4): ReLU()
        (5): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=256)
        (6): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
        (7): ReLU()
      )
      (linear): Linear(in_features=2560, out_features=1024, bias=True)
    )
    (encode_positions): ParakeetEncoderRelPositionalEncoding()
    (layers): ModuleList(
      (0-41): 42 x ParakeetEncoderBlock(
        (feed_forward1): ParakeetEncoderFeedForward(
          (linear1): Linear(in_features=1024, out_features=4096, bias=True)
          (activation): SiLUActivation()
          (linear2): Linear(in_fe

In [21]:
waveform, sample_rate = librosa.load(
    file_path,
    sr=None,
    mono=True
)

In [22]:
# resample to 16,000 hz
TARGET_SAMPLE_RATE = 16000
if sample_rate != TARGET_SAMPLE_RATE:
    waveform = librosa.resample(
        waveform,
        orig_sr = sample_rate,
        target_sr = TARGET_SAMPLE_RATE
    )
# normalizing audio amp
max_val = np.max(np.abs(waveform))
if max_val > 0:
    waveform /= max_val

In [24]:
# calc model inputs
model_inputs = processor(
    waveform,
    sampling_rate = TARGET_SAMPLE_RATE,
    return_tensors = 'pt'
)
model_inputs = model_inputs.to(
    device = model.device,
    dtype = model.dtype
)

In [25]:
# Verifying dtypes
print(model_inputs.input_features.dtype)
print(next(model.parameters()).dtype)

torch.bfloat16
torch.bfloat16


In [27]:
# obtain logits
with torch.no_grad():
    # Ensure input_features are in bfloat16 to match model's dtype
    model_inputs.input_features = model_inputs.input_features.to(torch.bfloat16)
    logits = model(**model_inputs).logits

In [28]:
# decode and translate logits to predicted text
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)

predicted_text = transcription[0]
print(predicted_text)

in your docket there is a record of attendance


In [34]:
# measuring accuracy with CER (character error rate) and WER (word error rate)

txt_file_path = f"/content/drive/MyDrive/Colab Notebooks/data/asr/{desired_file}.txt"
with open(txt_file_path, 'r') as file:
    ground_truth = [file.read().strip()]

print(f"WER: {wer(ground_truth, text)}")
print(f"CER: {cer(ground_truth, text)}")

WER: 0.0
CER: 0.0
